## Exercise List 6

#### 1. Utilizing Apache Spark and other correlated tools, beyond the graph constructed in Exercise List 5, implement the following options:

   - Determine the central node through the node degree.
   - Determine the central node by centrality utilizing the Wasserman distance.
   - Determine the central node by intermediation.

In [2]:
from pyspark import SparkConf
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from graphframes import *

In [5]:
conf = SparkConf().setAppName('appName').setMaster('local')
sc = SparkContext.getOrCreate(conf)
spark = SparkSession(sc)
vertexes = spark.read.csv('transport-nodes.csv', header=True)
edges = spark.read.csv('transport-relationships.csv', header=True)
graph = GraphFrame(vertexes, edges)

In [50]:
# Determinig the central node through its degree
graph_degree = graph.degrees
city = dict(degree=0, city_id='')
iter

for row_value in graph_degree.select("id", "degree").collect():
    if row_value.degree > city.get('degree'):
        city['degree'] = row_value.degree
        city['city_id'] = row_value.id
print(f"The central node is {city.get('city_id')}, with degree {city.get('degree')}")

The central node is Igarassu, with degree 4
